### A few examples of writing functions recursively along with a dynamic programming approach

In [33]:
# Function to comput the nth fibonacci number
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# This works and has O(2^n) runtime (dictated by the number of calls made)
print(fibonacci(6))

8


In [24]:
# Dynamic programming involves recording a cache of simpler results that can be used to solve more complex calls.
def dp_fibonacci(n):
    cache = [0, 1]
    
    if n <= len(cache):
        return cache[n]
    
    for i in range(2, n+1):
        cache.append(cache[i-2] + cache[i-1])
    return cache[-1]

In [31]:
# This has a runtime of O(n) so can calculate much larger values than the recursive approach
print(dp_fibonacci(6))
print(dp_fibonacci(6000))  # Recursive function for n=600 crashes my computer. Dynamic method finds the 6,000th one in a fraction of a second

8
14909059039666631258104


### Oh no! A child is running up n stairs and can take steps of either 1, 2 or 3. Calculate the number of possible ways the child can run up the stairs.

In [52]:
# If n=1, then the number of ways is 1
# If n=2, then the number of ways is 2: (1+1 steps, 2 steps)
# If n=3, then the number of ways is 4: (1+1+1, 1+2, 2+1, 3)

# If n=n+1 then the number of ways is the same as n steps + some extra amount corresponding to the last added step. This last
# step can be reached either with an additional step of size 1 from the nth configuration, additional steps of size (2, 1+1)
# from the n-2th configuration or additional steps of size (1+1+1, 1+2, 2+1, 3) from the n-3th configuation

# e.g. x[9] different ways of getting up the steps if n=9. If n=10, you have (x[9]+1) + (x[8]+2) + (x[7]+3)
# We know that x[0]=1, x[1]=1, x[2]=2, x[3]=4

def numberOfWays(n):
    cache = [1, 1, 2, 4]  # Initialise the base cases of there being 0, 1, 2 or 3 steps
    if n <= len(cache)-1:
        return cache[n]  # nth entry in the cache is the number of ways of getting up n stairs
    else:
        for n in range(4, n+1):
            #cache.append((cache[n-1]+1) + (cache[n-2]+2) + (cache[n-3]+4))
            cache.append(cache[n-1] + cache[n-2] + cache[n-3])
        return cache[-1]

print(numberOfWays(4))  # numberOfWays(10) = 274. Seems to work.

7


### Note the edited out line in the for loop. You don't need to add all the different ways of getting to the nth step from n-1th, n-2th and n-3th, you only need to add their raw value as this correponds to 'one way of getting to n from n-1 or n-2 or n-3'

### You don't need to add 1 or 2 or 4 because you'd be redundantly overcounting methods of getting to susbequent steps: for n=5, you can get there from n=4 with steps (1), or from n=3 with steps (1+1, 2) ... but the 1+1 takes you unavoidably to n=4 which has already been counted. You therefore only need to count the number to get to n-i without thinking about how many additional ways you can get there from prior steps

### Note further that the above isn't recursively calling itself which would result in an O(3^n) runtime (O(branches^depth)), instead by repeately calling the calculated numbers in cache, this is O(n)

### Oh no again! There's a robot in a grid with r rows and c columns. It can only move right or down, but some cells in the grid are off limits. Design an algorithm to navigate the robot from the top left to the bottom right

In [118]:
# Base case for this is a 2x2 grid with the robot in top left.
# From here, the robot can move either right or down, with the only applicable obstructions being located in the top right or
# lower left.

# There are 5(?) instances of the base case: with R denoting robot, and 0/1 denoting absence or presence of an obstruction we 
# could have [R,0;0,0], [R,1;0,0], [R,0;1,0], [R,0;0,1], [R,1;0,1]
# Edge cases for game over: [R,1;1,0], [R,1;1,1]

startingLocation = (0, 0)
x, y = startingLocation
r, c = 5, 5  # Initial grid parameters
obstructions = [(1,1), (2,2)]  # List of tuples describing the cells that are off limits

# Make a little grid so it's easier to see.
grid = [[0]*r for i in range(c)]
for obstruction in obstructions:
    grid[obstruction[0]][obstruction[1]] = 1
grid[0][0] = ord('R')  # ASCII code for letter R for robot.
grid

[[82, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

### So the robot wants to get to bottom right. Best strategy is probably to alternate steps, otherwise he could get stuck in edge cases when encountering obstructions either in the bottom row or last column

In [122]:
moves = ['rd', 'dr', 'rd', 'd', 'd']  # first 'd' corresponds to [R,0;0,1], because it just navigates you into [R,1;?,?]
configs = [[82, 0, 0, 0], [82, 1, 0, 0], [82, 0, 1, 0], [82, 0, 0, 1], [82, 1, 0, 1]]  # All possible configurations of the
# board that don't include edge cases

def returnState(x, y):
    return[82, grid[y][x+1], grid[y+1][x], grid[y+1][x+1]]  # Inspect the three grid neighbours

print(returnState(1,0))

[82, 0, 1, 0]


In [140]:
x, y = 0, 0
while (x <= r-1) and (y <= c-1):
    decision = configs.index(returnState(x,y))  # Index of the move in moves that needs to be executed
    print(returnState(x,y), decision)
    if moves[decision] == 'rd':
        print('Old x,y = {},{}'.format(x,y))
        x += 1
        y += 1
        print('New x,y = {},{}'.format(x,y))
    if moves[decision] == 'dr':
        print('Old x,y = {},{}'.format(x,y))
        y += 1
        x += 1
        print('New x,y = {},{}'.format(x,y))
    elif moves[decision] == 'd':
        print('Old x,y = {},{}'.format(x,y))
        y += 1
        print('New x,y = {},{}'.format(x,y))
    else:
        print('Might have crashed somewhere')

[82, 0, 0, 1] 3
Old x,y = 0,0
New x,y = 0,1
[82, 1, 0, 0] 1
Old x,y = 0,1
New x,y = 1,2
[82, 1, 0, 0] 1
Old x,y = 1,2
New x,y = 2,3
[82, 0, 0, 0] 0
Old x,y = 2,3
New x,y = 3,4
Might have crashed somewhere


IndexError: list index out of range

### Kind of working, but not really using dynamic programming, or preferentially trying to get to the lower right corner...

### Apparently better to start at the end and work backwards adding steps unless they're not allowed

In [141]:
def get_path(grid):
    if grid is None or len(grid) == 0:
        return None
    path = []
    failed = []
    if helper(grid, len(grid)-1, len(grid[0])-1, path, failed):
        return path
    return None
    
def helper(grid, row, col, path, failed):
    if row < 0 or col < 0 or grid[row][col] is None:
        return False
        
    point = (row,col)
    
    # Already visited this and failed!
    if point in failed:
        return False
        
    # If at origin then the path has made it all the way!
    if point == (0,0) or helper(grid, row-1, col, path, failed) or helper(grid, row, col-1, path, failed):
        path.append(point)
        return True
        
    failed.append(point)
    return False

In [143]:
grid = [[0, 0, 0, 0, 0, 0, None],
        [0, None, None, 0, None, None, 0],
        [0, 0, None, 0, 0, 0, 0],
        [None, None, 0, 0, 0, None, 0]]

print(get_path(grid))

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 3), (2, 3), (2, 4), (2, 5), (2, 6), (3, 6)]
